In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy import fftpack
from scipy import signal
import os

# 读图，转频域

In [ ]:
# 打开图片
img1 = Image.open('lena-512-gray.png').convert('L')
img1_array=np.array(img1)# 用fft-pack的内置函数
F = fftpack.fftn(img1_array)
F_magnitude = np.abs(fftpack.fftshift(F))
# 对数变换，使值域变为正值
F_magnitude_log = np.log1p(F_magnitude)

# 归一化到0-255
F_magnitude_rescale = 255 * (F_magnitude_log - F_magnitude_log.min()) / (F_magnitude_log.max() - F_magnitude_log.min())

# 转为8位无符号整数
F_magnitude_uint8 = np.uint8(F_magnitude_rescale)

# 保存频谱图
# 转为Pillow图像并保存
Image.fromarray(F_magnitude_uint8).save('./assignment2/lena-512-freq.png')

# 非理想滤波器


In [ ]:
# 绘制频率响应
def plot_horizontal_slice(mask, title='Horizontal Slice through the Center'):
    center_row = mask.shape[0] // 2
    plt.figure()
    plt.plot(mask[center_row, :])
    plt.title(title)
    plt.xlabel('Frequency')
    plt.ylabel('Amplitude')
    plt.grid(True)
    plt.show()

# Example usage:
img_shape = (256, 256)  # Example image shape

# 1、巴特沃斯滤波器
<sup>低通：</sup>
$H(u,v) = \frac{1}{1 + \left(\frac{D(u,v)}{D0}\right)^{2n}}$
<sup>高通：</sup>
$H(u,v) = \frac{1}{1 + \left(\frac{D0}{D(u,v)}\right)^{2n}}$
<sup>带通：</sup>
$H(u,v) = \frac{1}{1 + \left(\frac{D(u,v)}{D0}\right)^{2n}} \times \frac{1}{1 + \left(\frac{D1}{D(u,v)}\right)^{2n}}$

In [ ]:
# 非理想滤波器
# Butterworth
# lowpass
def butterworth_lowpass_filter(img, cutoff, order):
    m, n = img.shape
    u = np.arange(m) - m // 2
    v = np.arange(n) - n // 2
    U, V = np.meshgrid(u, v)
    D = np.sqrt(U**2 + V**2)+1e-5
    mask = 1 / (1 + (D / cutoff) ** (2 * order))
    return mask

# highpass
def butterworth_highpass_filter(img, cutoff, order):
    m, n = img.shape
    u = np.arange(m) - m // 2
    v = np.arange(n) - n // 2
    U, V = np.meshgrid(u, v)
    D = np.sqrt(U**2 + V**2)+1e-5
    mask = 1 / (1 + (cutoff / D) ** (2 * order))
    return mask

# bandpass
def butterworth_bandpass_filter(img, low_cutoff, high_cutoff, order):
    return butterworth_lowpass_filter(img, high_cutoff, order) * butterworth_highpass_filter(img, low_cutoff, order)


In [ ]:
# 频率响应
cutoff_low = 20
cutoff_high = 50
order = 5  # Higher order for steeper transition

# Generate the masks
lowpass_mask = butterworth_lowpass_filter(np.zeros(img_shape), cutoff_low, order)

# Plot horizontal and vertical slices
plot_horizontal_slice(lowpass_mask, title='Horizontal Slice of Butterworth Low-pass Filter (Order {})'.format(order))

<sup>2、高斯滤波器</sup>
<sup>低通：</sup>
$H(u,v) = e^{\frac{-D(u,v)^{2}}{2D_{0}^{2}}}$
<sup>高通：</sup>
$H(u,v) = 1 - e^{\frac{-D(u,v)^{2}}{2D_{0}^{2}}}$
<sup>带通：</sup>
$H(u,v) = e^{\frac{-D(u,v)^{2}}{2D_{1}^{2}}} - e^{\frac{-D(u,v)^{2}}{2D_{0}^{2}}}$

In [ ]:
# Gauss
# lowpass
def gaussian_lowpass_filter(img, cutoff):
    m, n = img.shape
    u = np.arange(m) - m // 2
    v = np.arange(n) - n // 2
    U, V = np.meshgrid(u, v)
    D = np.sqrt(U**2 + V**2)
    mask = np.exp(-D**2 / (2 * cutoff**2))
    return mask

# highpass
def gaussian_highpass_filter(img, cutoff):
    m, n = img.shape
    u = np.arange(m) - m // 2
    v = np.arange(n) - n // 2
    U, V = np.meshgrid(u, v)
    D = np.sqrt(U**2 + V**2)
    mask = 1 - np.exp(-D**2 / (2 * cutoff**2))
    return mask

# bandpass
def gaussian_bandpass_filter(img, low_cutoff, high_cutoff):
    m, n = img.shape
    u = np.arange(m) - m // 2
    v = np.arange(n) - n // 2
    U, V = np.meshgrid(u, v)
    D = np.sqrt(U**2 + V**2)
    mask_low = np.exp(-D**2 / (2 * low_cutoff ** 2))
    mask_high = 1 - np.exp(-D**2 / (2 * high_cutoff ** 2))
    mask = mask_high - mask_low
    return mask

In [ ]:
# 频率响应
cutoff_low = 20
order = 5  # Higher order for steeper transition

# Generate the masks
lowpass_mask = gaussian_lowpass_filter(np.zeros(img_shape), cutoff_low)


# Plot horizontal and vertical slices
plot_horizontal_slice(lowpass_mask, title='Horizontal Slice of Gaussian Low-pass Filter (Order {})'.format(order))

<sup>3、椭圆滤波器</sup>
椭圆滤波器的模拟传递函数：
$\[H(s) = \frac{P(s)}{Q(s)}\]$

数字椭圆滤波器的传递函数：
$\[H(z) = \frac{B(z)}{A(z)}\]$


In [ ]:
def elliptic_lowpass_filter(img, rp, rs, cutoff, order=5):
    m, n = img.shape
    u = np.arange(m) - m // 2
    v = np.arange(n) - n // 2
    U, V = np.meshgrid(u, v)
    D = np.sqrt(U**2 + V**2)

    # Normalize the frequency
    D_norm = D / np.max(D)

    # Compute the filter coefficients
    b, a = signal.ellip(order, rp, rs, cutoff, 'low', analog=False)  # Set analog to False for digital filter

    # Evaluate the elliptic rational function
    w, h = signal.freqz(b, a, worN=D_norm.ravel(), fs=2)  # Use freqz for digital filter
    h = h.reshape(D_norm.shape)
    return np.abs(h)

def elliptic_highpass_filter(img, rp, rs, cutoff, order=5):
    m, n = img.shape
    u = np.arange(m) - m // 2
    v = np.arange(n) - n // 2
    U, V = np.meshgrid(u, v)
    D = np.sqrt(U**2 + V**2)

    # Normalize the frequency
    D_norm = D / np.max(D)

    # Compute the filter coefficients
    b, a = signal.ellip(order, rp, rs, cutoff, 'high', analog=False)  # Set analog to False for digital filter

    # Evaluate the elliptic rational function
    w, h = signal.freqz(b, a, worN=D_norm.ravel(), fs=2)  # Use freqz for digital filter
    h = h.reshape(D_norm.shape)
    return np.abs(h)

def elliptic_bandpass_filter(img, rp, rs, low_cutoff,high_cutoff, order=5):
    m, n = img.shape
    u = np.arange(m) - m // 2
    v = np.arange(n) - n // 2
    U, V = np.meshgrid(u, v)
    D = np.sqrt(U**2 + V**2)

    # Normalize the frequency
    D_norm = D / np.max(D)

    # Compute the filter coefficients
    b, a = signal.ellip(order, rp, rs, [low_cutoff, high_cutoff], 'band', analog=False)  # Set analog to False for digital filter

    # Evaluate the elliptic rational function
    w, h = signal.freqz(b, a, worN=D_norm.ravel(), fs=2)  # Use freqz for digital filter
    h = h.reshape(D_norm.shape)
    return np.abs(h)

In [ ]:
# 频率响应
rp = 1        # Ripple in the passband (dB)
rs = 40       # Attenuation in the stopband (dB)
cutoff = 0.1  # Cutoff frequency as a fraction of the Nyquist frequency
order = 5    # Order of the elliptic filter

# Create elliptic lowpass filter mask
elliptic_lowpass_mask = elliptic_lowpass_filter(np.zeros(img_shape), rp, rs, cutoff, order)
elliptic_highpass_mask = elliptic_highpass_filter(np.zeros(img_shape), rp, rs, cutoff, order)
elliptic_bandpass_mask = elliptic_bandpass_filter(np.zeros(img_shape), rp, rs,0.1,0.5, order)
# Plot horizontal and vertical slices through the mask
plot_horizontal_slice(elliptic_lowpass_mask, title='Horizontal Slice of Elliptic Low-pass Filter (Order {})'.format(order))
plot_horizontal_slice(elliptic_highpass_mask, title='Horizontal Slice of Elliptic High-pass Filter (Order {})'.format(order))
plot_horizontal_slice(elliptic_bandpass_mask, title='Horizontal Slice of Elliptic Band-pass Filter (Order {})'.format(order))

In [ ]:
def apply_filter(img,filter_type,band=False, *args, **kwargs):
    print(kwargs)
    filter=globals()[filter_type]
    mask=filter(img,*args,**kwargs)
    F_filtered = F * fftpack.fftshift(mask)
    img_filtered = np.abs(fftpack.ifftn(F_filtered))
    img_pass = Image.fromarray(img_filtered.astype('uint8'))
    img_pass.show()
    directory = './assignment2/' + str(filter_type)
    # 检查文件夹是否存在，如果不存在则创建它
    if not os.path.exists(directory):
        os.makedirs(directory)

    # 保存图像
    if band:
        # 获取 low 和 high 参数的值
        low = kwargs.get('low_cutoff')
        high = kwargs.get('high_cutoff')
        img_pass.save(os.path.join(directory, f'lena-512-{str(filter_type.split("_")[1])}_{low}-{high}.png'))
    else:
        img_pass.save(os.path.join(directory, f'lena-512-{str(filter_type.split("_")[1])}_{args[-1]}.png'))
    return []

In [ ]:
for low_cutoff in range(10,100,10):
    apply_filter(img1_array,'butterworth_lowpass_filter',low_cutoff,order=5)
    apply_filter(img1_array,'gaussian_lowpass_filter',low_cutoff)
    apply_filter(img1_array,'elliptic_lowpass_filter',1,40,low_cutoff/100,order=5)

In [ ]:
for high_cutoff in range(10,100,10):
    apply_filter(img1_array,'butterworth_highpass_filter',high_cutoff,order=5)
    apply_filter(img1_array,'gaussian_highpass_filter',high_cutoff)
    apply_filter(img1_array,'elliptic_highpass_filter',1,40,high_cutoff/100,order=5)

In [ ]:
for low_cutoff in range(20,81,20):
    for high_cutoff in range(low_cutoff+20,low_cutoff+81,20):
        apply_filter(img=img1_array, band=True, filter_type='butterworth_bandpass_filter', low_cutoff=low_cutoff, high_cutoff=high_cutoff, order=5)
        apply_filter(img=img1_array, band=True, filter_type='gaussian_bandpass_filter', low_cutoff=low_cutoff, high_cutoff=high_cutoff)
        apply_filter(img=img1_array, band=True, filter_type='elliptic_bandpass_filter', rp=1, rs=40, low_cutoff=low_cutoff/100, high_cutoff=high_cutoff/100, order=5)
